# zawęzanie okna
## nazwa funkcji: limiter(DF,name,main_category,category,goal_usd,duration,country,currency)
#### argumenty (input): 
* DF - data frame do ucięcia
* name - nazwa kampanii
* main category - wiadomo
* category - wiadomo
* goal - wnioskowana kwota, przeliczona na dolary
* duration - czas trwania kampanii
* country - wiadomo
* currency - wiadomo



#### output: DF_limited

#### założenia
* na podstawie wprowadzonych danych funkcja ma za zadanie ograniczyć rozmiar dataframe'u do projektów, które są "podobne" do ocenianego przez apkę 
* TYMCZASOWO - funkcja będzie zawężać pełny data frame, otwarty tutaj, docelowo będzie korzystać z data frame otwartego przez inny kawałek apki
* TYMCZASOWO - koszyki dla wnioskowanej kwoty, czasu trwania kampanii i krajów powyznaczam "jakoś" na podstawie notebooka z analizami, doszlifuję je w dalszym etapie w razie potrzeby
* SPECJALNA WARTOŚĆ DLA INPUTÓW: 'not considered', stosowana gdy nie chcemy za bardzo zawężać 


## do zrobienia:
* zakodowanie koszyków zgodnie z wynikami analizy NU/PA

## dodatki / pomysły: 
* napisanie generatora losowych danych testowych? - BS?
* słowa kluczowe - przeszukanie datasetu pod kątem słów najczęściej występujących w nazwach projektów (np. w danej kategorii), które odniosły sukces i ewentualne uwzględnienie takich słów, jeśli występują w naszej nazwie?
* podejście alternatywne - sprawdzenie najczęściej występujących w nazwach słów i określenie "skuteczności" projektów zawierających dane słowo? (temu pomysłowi brakuje czegoś..)


* przygotowanie odpowiedniego inputu - PA?
* po outpucie, ewentualny powrót z rozszerzeniem na kategorie lub waluty (lub inne - do dodania bez problemu) - MZ?

### DF

In [1]:
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt

file_path = (glob.glob(os.path.join(os.path.abspath(''), '**', '*kickstarter_filtered*.tsv'), recursive=True))

DF = pd.read_csv(file_path[0], header = 0, sep='\t')
DF


,id,name,main_category,category,country,launched,deadline,duration,currency,goal_in_defined_currency,pledged_in_defined_currency,goal_in_usd,pledged_in_usd,percentage_of_money_collected,backers,state
0,1860890148,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,United States,2009-04-21,2009-05-31,40,USD,1000.0,625.0,1000.00,625.00,0.625000,30,failed
1,709707365,CRYSTAL ANTLERS UNTITLED MOVIE,Film & Video,Shorts,United States,2009-04-23,2009-07-20,88,USD,80000.0,22.0,80000.00,22.00,0.000275,3,failed
2,1703704063,drawing for dollars,Art,Illustration,United States,2009-04-24,2009-05-03,9,USD,20.0,35.0,20.00,35.00,1.750000,3,successful
3,727286,Offline Wikipedia iPhone app,Technology,Software,United States,2009-04-25,2009-07-14,80,USD,99.0,145.0,99.00,145.00,1.464646,25,successful
4,1622952265,Pantshirts,Fashion,Fashion,United States,2009-04-27,2009-05-26,29,USD,1900.0,387.0,1900.00,387.00,0.203684,10,failed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368233,1315429286,Healthcare Web TV Network,Journalism,Web,United States,2017-12-29,2018-01-18,20,USD,100000.0,0.0,100000.00,0.00,0.000000,0,canceled
368234,801423615,Collaboration with Snoop Dogg,Music,Hip-Hop,Australia,2017-12-29,2018-01-01,3,AUD,500.0,655.0,392.06,513.60,1.310004,6,successful
368235,1057488050,15 Games,Games,Tabletop Games,United States,2017-12-31,2018-01-05,5,USD,1000.0,1.0,1000.00,1.00,0.001000,1,canceled
368236,1854247643,100 Acres of Hell,Comics,Comic Books,Great Britain (UK),2018-01-01,2018-01-15,14,GBP,100.0,57.0,136.40,77.75,0.570015,6,canceled


### Koszyki

In [2]:
#print(DF['goal_in_usd'].describe())

#wartości i ilość koszyków z czapki

def basket_goal(goal):
    if goal <= 500:
        goal_min = 0
        goal_max = 600
    elif goal <= 1000:
        goal_min = 300
        goal_max = 1200
    elif goal <= 2000:
        goal_min = 800
        goal_max = 2500
    elif goal <= 4000:
        goal_min = 1500
        goal_max = 5000
    elif goal <= 10000:
        goal_min = 4000
        goal_max = 12000
    elif goal <= 20000:
        goal_min = 8000
        goal_max = 25000
    elif goal <= 40000:
        goal_min = 16000
        goal_max = 50000
    elif goal < 100000:
        goal_min = 30000
        goal_max = 120000
    else:
        goal_min = 80000
        goal_max = 20000000
    return[goal_min, goal_max]


In [3]:
# czas trwania koszyki

def basket_dur(dur):
    if dur < 30:
        dur_min = 0
        dur_max = 29
    elif dur == 30:
        dur_min = 30
        dur_max = 30
    else:
        dur_min = 31
        dur_max = 100
    return[dur_min, dur_max]

In [4]:
#print(DF['goal_in_usd'].describe())
europe_list = [#'Australia',
                     'Austria',
                     'France',
                     'Switzerland',
                     'Belgium',
                     'Denmark',
                     #'Canada',
                     #'Singapore',
                     #'New Zealand',
                     #'Hong Kong',
                     'Sweden',
                     'Germany',
                     'Ireland',
                     'Luxembourg',
                     #'Japan',
                     'Netherlands',
                     'Italy',
                     'Great Britain (UK)',
                     'Spain',
                     #'United States',
                     #'Mexico',
                     'Norway']
other_country_list = ['Australia',
                     #'Austria',
                     #'France',
                     #'Switzerland',
                     #'Belgium',
                     #'Denmark',
                     'Canada',
                     'Singapore',
                     'New Zealand',
                     'Hong Kong',
                     #'Sweden',
                     #'Germany',
                     #'Ireland',
                     #'Luxembourg',
                     'Japan',
                     #'Netherlands',
                     #'Italy',
                     #'Great Britain (UK)',
                     #'Spain',
                     #'United States',
                     'Mexico',
                     #'Norway'
                     ]

def basket_country(country):
    if country == 'United States':
        return ['United States']
    elif country in europe_list:
        return europe_list
    elif country in other_country_list:
        return other_country_list
    else: 
        return other_country_list
        
#list(set(DF['country'].to_numpy()))


In [5]:
# koszykownik dla kategorii nieskończony, względnie dobrane koszyki tylko dla main category Technology

def basket_cat(mcat,cat):
    if mcat == 'Art':
        if cat in ['Sculpture',
                 'Conceptual Art',
                 'Illustration',
                 'Public Art',
                 'Mixed Media',
                 'Video Art',
                 'Art',
                 'Installations',
                 'Ceramics',
                 'Textiles',
                 'Painting',
                 'Digital Art',
                 'Performance Art']:
            return(['Sculpture',
                 'Conceptual Art',
                 'Illustration',
                 'Public Art',
                 'Mixed Media',
                 'Video Art',
                 'Art',
                 'Installations',
                 'Ceramics',
                 'Textiles',
                 'Painting',
                 'Digital Art',
                 'Performance Art'])
    elif mcat == 'Comics':
        if cat in ['Comic Books',
                 'Events',
                 'Anthologies',
                 'Graphic Novels',
                 'Comics',
                 'Webcomics']:
            return(['Comic Books',
                 'Events',
                 'Anthologies',
                 'Graphic Novels',
                 'Comics',
                 'Webcomics'])
    elif mcat == 'Crafts':
        if cat in ['Weaving',
                 'Letterpress',
                 'Crochet',
                 'Candles',
                 'Taxidermy',
                 #'Crafts',
                 'Knitting',
                 'Embroidery',
                 'Pottery',
                 'Glass',
                 'Quilts',
                 'Printing',
                 'DIY',
                 'Stationery',
                 'Woodworking']:
            return(['Weaving',
                 'Letterpress',
                 'Crochet',
                 'Candles',
                 'Taxidermy',
                 #'Crafts',
                 'Knitting',
                 'Embroidery',
                 'Pottery',
                 'Glass',
                 'Quilts',
                 'Printing',
                 'DIY',
                 'Stationery',
                 'Woodworking'])
        elif cat in ['Crafts']:
            return(['Crafts'])
    elif mcat == 'Dance':
        if cat in ['Dance', 'Spaces', 'Residencies', 'Workshops', 'Performances']:
            return(['Dance', 'Spaces', 'Residencies', 'Workshops', 'Performances'])
    elif mcat == 'Design':
        if cat in ['Graphic Design',
                 'Product Design',
                 'Civic Design',
                 'Interactive Design',
                 'Architecture',
                 'Typography',
                 'Design']:
            return(['Graphic Design',
                 'Product Design',
                 'Civic Design',
                 'Interactive Design',
                 'Architecture',
                 'Typography',
                 'Design'])
    elif mcat == 'Fashion':
        if cat in ['Couture',
                 'Childrenswear',
                 'Footwear',
                 'Fashion',
                 'Accessories',
                 'Jewelry',
                 'Apparel',
                 'Ready-to-wear',
                 'Pet Fashion']:
            return(['Couture',
                 'Childrenswear',
                 'Footwear',
                 'Fashion',
                 'Accessories',
                 'Jewelry',
                 'Apparel',
                 'Ready-to-wear',
                 'Pet Fashion'])
    elif mcat == 'Film & Video':
        if cat in ['Television',
                 'Horror',
                 'Film & Video',
                 'Science Fiction',
                 'Music Videos',
                 'Documentary',
                 'Family',
                 'Animation',
                 'Fantasy',
                 'Experimental',
                 'Movie Theaters',
                 'Comedy',
                 'Webseries',
                 'Festivals',
                 'Romance',
                 'Drama',
                 'Shorts',
                 'Narrative Film',
                 'Thrillers',
                 'Action']:
            return(['Television',
                 'Horror',
                 'Film & Video',
                 'Science Fiction',
                 'Music Videos',
                 'Documentary',
                 'Family',
                 'Animation',
                 'Fantasy',
                 'Experimental',
                 'Movie Theaters',
                 'Comedy',
                 'Webseries',
                 'Festivals',
                 'Romance',
                 'Drama',
                 'Shorts',
                 'Narrative Film',
                 'Thrillers',
                 'Action'])
    elif mcat == 'Food':
        if cat in ['Spaces',
                 'Vegan',
                 'Drinks',
                 'Restaurants',
                 'Food',
                 'Events',
                 'Farms',
                 'Cookbooks',
                 "Farmer's Markets",
                 'Food Trucks',
                 'Community Gardens',
                 'Small Batch',
                 'Bacon']:
            return(['Spaces',
                 'Vegan',
                 'Drinks',
                 'Restaurants',
                 'Food',
                 'Events',
                 'Farms',
                 'Cookbooks',
                 "Farmer's Markets",
                 'Food Trucks',
                 'Community Gardens',
                 'Small Batch',
                 'Bacon'])
    elif mcat == 'Games':
        if cat in ['Playing Cards',
                 'Puzzles',
                 'Games',
                 'Mobile Games',
                 'Gaming Hardware',
                 'Video Games',
                 'Tabletop Games',
                 'Live Games']:
            return(['Playing Cards',
                 'Puzzles',
                 'Games',
                 'Mobile Games',
                 'Gaming Hardware',
                 'Video Games',
                 'Tabletop Games',
                 'Live Games'])
    elif mcat == 'Journalism':
        if cat in ['Print', 'Video', 'Journalism', 'Web', 'Photo', 'Audio']:
            return(['Print', 'Video', 'Journalism', 'Web', 'Photo', 'Audio'])
    elif mcat == 'Music':
        if cat in ['Classical Music',
                 'Faith',
                 'Chiptune',
                 'Blues',
                 'Indie Rock',
                 'Pop',
                 'Kids',
                 'Hip-Hop',
                 'World Music',
                 'Jazz',
                 'Latin',
                 'Metal',
                 'Comedy',
                 'Country & Folk',
                 'R&B',
                 'Rock',
                 'Music',
                 'Punk',
                 'Electronic Music']:
            return(['Classical Music',
                 'Faith',
                 'Chiptune',
                 'Blues',
                 'Indie Rock',
                 'Pop',
                 'Kids',
                 'Hip-Hop',
                 'World Music',
                 'Jazz',
                 'Latin',
                 'Metal',
                 'Comedy',
                 'Country & Folk',
                 'R&B',
                 'Rock',
                 'Music',
                 'Punk',
                 'Electronic Music'])
    elif mcat == 'Photography':
        if cat in ['Photobooks',
                 'Animals',
                 'Places',
                 'Nature',
                 'People',
                 'Fine Art',
                 'Photography']:
            return(['Photobooks',
                 'Animals',
                 'Places',
                 'Nature',
                 'People',
                 'Fine Art',
                 'Photography'])
    elif mcat == 'Publishing':
        if cat in ['Radio & Podcasts',
                 'Art Books',
                 'Literary Journals',
                 'Poetry',
                 'Calendars',
                 'Comedy',
                 'Letterpress',
                 'Literary Spaces',
                 "Children's Books",
                 'Zines',
                 'Academic',
                 'Publishing',
                 'Anthologies',
                 'Periodicals',
                 'Nonfiction',
                 'Fiction',
                 'Translations',
                 'Young Adult']:
            return(['Radio & Podcasts',
                 'Art Books',
                 'Literary Journals',
                 'Poetry',
                 'Calendars',
                 'Comedy',
                 'Letterpress',
                 'Literary Spaces',
                 "Children's Books",
                 'Zines',
                 'Academic',
                 'Publishing',
                 'Anthologies',
                 'Periodicals',
                 'Nonfiction',
                 'Fiction',
                 'Translations',
                 'Young Adult'])
    elif mcat == 'Technology':
        if cat in [#'Technology',
                 'Wearables',
                 #'Apps',
                 'Space Exploration',
                 'Sound',
                 #'Gadgets',
                 #'Software',
                 '3D Printing',
                 'Robots',
                 'DIY Electronics',
                 #'Hardware',
                 #'Web',
                 'Makerspaces',
                 'Camera Equipment',
                 'Flight',
                 'Fabrication Tools']:
            return([#'Technology',
                 'Wearables',
                 #'Apps',
                 'Space Exploration',
                 'Sound',
                 #'Gadgets',
                 #'Software',
                 '3D Printing',
                 'Robots',
                 'DIY Electronics',
                 #'Hardware',
                 #'Web',
                 'Makerspaces',
                 'Camera Equipment',
                 'Flight',
                 'Fabrication Tools'])
        else:
            return([cat])
    elif mcat == 'Theater':
        if cat in ['Spaces',
                 'Comedy',
                 'Festivals',
                 'Experimental',
                 'Musical',
                 'Theater',
                 'Plays',
                 'Immersive']:
            return(['Spaces',
                 'Comedy',
                 'Festivals',
                 'Experimental',
                 'Musical',
                 'Theater',
                 'Plays',
                 'Immersive'])


#list(set(DF[DF['main_category'] == 'Theater']['category'].to_numpy()))


###  Wymagane argumenty dla funkcji - docelowo importowane z innego notebooka

In [6]:
pr_name = '3 axis video stabilizer for cell phones'
pr_main_category = 'Technology'
#pr_category = 'Gadgets'
pr_category = 'not considered'
pr_goal_usd = 700
pr_duration = 30
pr_country = 'United States'
pr_currency = 'USD'
#r_currency = 'not considered'

### Ograniczanie wielkości ramki


In [7]:
def limiter(df,name,mcat,cat,goal,dur,country,curr):

    goal_min = basket_goal(goal)[0]
    goal_max = basket_goal(goal)[1]
    war_goal_min = df['goal_in_usd'] >= goal_min
    war_goal_max = df['goal_in_usd'] <= goal_max
    
    war_mcat = df['main_category'] == mcat
    
    dur_min = basket_dur(dur)[0]
    dur_max = basket_dur(dur)[1]
    war_dur_min = df['duration'] >= dur_min
    war_dur_max = df['duration'] <= dur_max
    
    country_list = basket_country(country)
    war_country = df['country'].isin(country_list)
    #print(country_list)
    
    if cat != 'not considered':
        cat_list = basket_cat(mcat,cat)
        war_cat_list = df['category'].isin(cat_list)
    else:
        war_cat_list = df['category'] != ''
    #print(war_cat_list)
    
    
    if curr == 'not considered':
        war_curr = df['currency'] != ''
    elif curr == 'USD':
        war_curr = df['currency'] == 'USD'
    else:
        war_curr = df['currency'] != 'USD'
    
    
    return (df.loc[war_cat_list][(war_goal_min) & (war_goal_max) & (war_mcat) & (war_dur_max) & (war_dur_min) & (war_curr) & (war_country)])

DF_limited = limiter(DF,pr_name,pr_main_category,pr_category,pr_goal_usd,pr_duration,pr_country,pr_currency)

DF_limited

    

,id,name,main_category,category,country,launched,deadline,duration,currency,goal_in_defined_currency,pledged_in_defined_currency,goal_in_usd,pledged_in_usd,percentage_of_money_collected,backers,state
365,1451429380,Subtitles in Miro - Translations and Support f...,Technology,Software,United States,2009-09-01,2009-10-01,30,USD,1000.0,1002.00,1000.0,1002.00,1.002000,26,successful
721,1249317992,Fabroot - Making 3D Fabrication Available to E...,Technology,Hardware,United States,2009-10-02,2009-11-01,30,USD,1100.0,175.00,1100.0,175.00,0.159091,7,failed
3925,1825843719,Friends With Websites,Technology,Software,United States,2010-05-31,2010-06-30,30,USD,300.0,0.00,300.0,0.00,0.000000,0,failed
5905,1382999730,Look at how Kickstarter works,Technology,Software,United States,2010-07-27,2010-08-26,30,USD,1000.0,0.00,1000.0,0.00,0.000000,0,failed
9308,1702131372,plode.me server costs,Technology,Software,United States,2010-11-02,2010-12-02,30,USD,500.0,20.00,500.0,20.00,0.040000,1,failed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364503,697802471,Pulse,Technology,Apps,United States,2017-11-06,2017-12-06,30,USD,1000.0,405.00,1000.0,405.00,0.405000,8,failed
365002,973232248,Best phone and tablet holder,Technology,Gadgets,United States,2017-11-09,2017-12-09,30,USD,500.0,975.00,500.0,975.00,1.950000,80,successful
365086,2063306371,AR4X - The Intelligent Camera Robot that Redef...,Technology,DIY Electronics,United States,2017-11-10,2017-12-10,30,USD,1000.0,6221.00,1000.0,6221.00,6.221000,21,successful
365666,571709005,"RUGGED | Off-Road, All Terrain, Any Weather He...",Technology,Wearables,United States,2017-11-14,2017-12-14,30,USD,1000.0,138120.98,1000.0,138120.98,138.120980,571,successful
